In [47]:

import os
import time

import pandas as pd
import requests
import torch
import torch.nn as nn
from bs4 import BeautifulSoup as Soup
from scipy.special import style
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from torch.cuda import device
from torch.distributed.pipelining import pipeline
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import pipeline

### Login + Page

In [8]:
# 抓取貼文內容--------------------
browser = webdriver.Chrome()
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("14han_15felix_")
password_input.send_keys("Yaya0329@Instagram")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(20)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 
subUrlList = ['wendys', 'sonicdrivein', 'mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada']

for subUrl in subUrlList:
    # 前往頁面
    browser.get(url + '/' + subUrl)
    time.sleep(2)
    # 往下滑並取得新的貼文連結
    post_all = []
    postList = []

    # Get scroll height.
    last_height = browser.execute_script("return document.body.scrollHeight")
    with tqdm(total=100) as pbar:
        while True:
            # Scroll down to the bottom.
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load the page.
            time.sleep(5)
    
            soup = Soup(browser.page_source, 'lxml')
            # 尋找所有的貼文連結
        
            for rowPost in soup.select('div._ac7v'):
                for elem in rowPost.select('div a'):
                    if elem['href'] in postList:
                        continue
                    postList.append(elem['href'])
                    img_elem = elem.select('div div img')
                    temp = []
                    temp.append(elem['href'])
                    try: 
                        temp.append(img_elem[0]['alt'])
                    except:
                        temp.append("")
                    temp.append(img_elem[0]['src'])
                    post_all.append(temp)
                    
            # Calculate new scroll height and compare with last scroll height.
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            pbar.update(1)
        
    print(subUrl+" 總共 " + str(len(post_all)) + " 篇貼文")
    
    # 存成csv
    postDataframe= pd.DataFrame(post_all)
    postDataframe.rename(columns = {0:'url', 1:'imgAlt', 2:'imgSrc'}, inplace = True)
    postDataframe.to_csv('Home_'+subUrl+'.csv', index = False)

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


inputing username and password...


KeyboardInterrupt: 

In [2]:
# 抓取貼文內容--------------------
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=334,778")
browser = webdriver.Chrome(options=chrome_options)
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("_blood_mango_")
password_input.send_keys("wangzeng1")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(15)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 
subUrlList = ['sonicdrivein']
# 'wendys', 'mcdonalds', 'mcdonalds_switzerland', 'mcdonaldscanada', 'mcdonalds_switzerland'
# ,'sonicdrivein'

for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    # insHome = pd.read_csv('Home_'+subUrl+'.csv')
    insHome = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    with tqdm(total=insHome.shape[0]) as pbar:
        counter = 0
        for index, row in insHome.iterrows(): #< 922
            if '/p/' in row['url'] and counter > 406 and counter < 923:
                browser.get(url + row['url'])
                time.sleep(10)
                html = browser.page_source
                soup = Soup(html, 'lxml')
                img_elem = soup.select('img.x5yr21d')
                text_elem = soup.select('h1._ap3a')

                insHome.at[index, 'imgSrc'] = img_elem[1]['src']
                try:
                    insHome.at[index, 'imgAlt'] = text_elem[0].text
                except:
                    insHome.at[index, 'imgAlt'] = ""
                insHome.to_csv('Done_'+subUrl+'.csv', index = False)
            pbar.update(1)
            counter = counter+1
            
    insHome['imgName'] = subUrl+'_'+insHome.index.astype(str)
    insHome.to_csv('Done_'+subUrl+'.csv', index = False)

inputing username and password...
login successfully
parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done


100%|██████████| 2087/2087 [1:20:06<00:00,  2.30s/it] 


### Download Image

In [ ]:
subUrlList = ['wendys','mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada','sonicdrivein']

for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    insDone = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    with tqdm(total=insDone.shape[0]) as pbar:
        for index, row in insDone.iterrows():
            #create img folder
            if not os.path.exists(subUrl+'_img'):
                os.makedirs(subUrl+'_img')
            with open(subUrl+'_img/'+row['imgName']+'.jpg', 'wb') as f:
                try :
                    f.write(requests.get(row['imgSrc']).content)
                except:
                    print('Failed to download' + row['imgName']+'.jpg')
            pbar.update(1)

parsing wendys.csv
parsing wendys.csv done


100%|██████████| 371/371 [00:06<00:00, 54.63it/s]


parsing mcdonalds.csv
parsing mcdonalds.csv done


100%|██████████| 307/307 [00:10<00:00, 28.52it/s]


parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done


...

100%|██████████| 850/850 [00:15<00:00, 53.73it/s]


parsing sonicdrivein.csv
parsing sonicdrivein.csv done


 23%|██▎       | 514/2240 [01:42<06:35,  4.37it/s]

Failed to downloadsonicdrivein_513.jpg


 78%|███████▊  | 1757/2240 [06:08<03:22,  2.38it/s]

In [33]:
# 抓取貼文內容--------------------
browser = webdriver.Chrome()
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("_wang_zeng")
password_input.send_keys("wangzeng1")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(20)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 
subUrlList = ['wendys', 'sonicdrivein', 'mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada']
post_count=[371, 2240, 307, 2087, 850]
counter = -1
for subUrl in subUrlList:
    download_counter = 0
    counter = counter+1
    print("parsing "+subUrl+".csv")
    insDone = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    
    # 前往頁面
    browser.get(url + '/' + subUrl)
    time.sleep(2)
    # 往下滑並取得新的貼文連結
    post_all = []
    postList = []

    # Get scroll height.
    last_height = browser.execute_script("return document.body.scrollHeight")
    with tqdm(total=post_count[counter]) as pbar:
        while True:
            # Scroll down to the bottom.
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load the page.
            time.sleep(5)
    
            soup = Soup(browser.page_source, 'lxml')
            # 尋找所有的貼文連結
        
            for rowPost in soup.select('div._ac7v'):
                for elem in rowPost.select('div a'):
                    if elem['href'] in postList:
                        continue
                        
                    imgName = insDone[insDone['url'] == elem['href']]['imgName']
                    if imgName.shape[0] > 0:
                        img_elem = elem.select('div div img')
                        imgUrl = img_elem[0]['src']
                        if not os.path.exists(subUrl+'_img'):
                            os.makedirs(subUrl+'_img')
                        with open(subUrl+'_img/'+imgName.values[0]+'.jpg', 'wb') as f:
                            try :
                                f.write(requests.get(imgUrl).content)
                                download_counter = download_counter+1
                            except:
                                print('Failed to download' + imgName+'.jpg')
                        postList.append(elem['href'])
                    pbar.update(1)
            # Calculate new scroll height and compare with last scroll height.
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    print(subUrl+" 總共 " + str(download_counter) + " 篇圖片已下載")
    

inputing username and password...
login successfully
parsing wendys.csv
parsing wendys.csv done


377it [02:55,  2.15it/s]                         


wendys 總共 371 篇圖片已下載
parsing sonicdrivein.csv
parsing sonicdrivein.csv done


2301it [17:40,  2.17it/s]                          


sonicdrivein 總共 2219 篇圖片已下載
parsing mcdonalds.csv
parsing mcdonalds.csv done


...

mcdonalds 總共 307 篇圖片已下載
parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done


2202it [16:28,  2.23it/s]                          


mcdonalds_switzerland 總共 2050 篇圖片已下載
parsing mcdonaldscanada.csv
parsing mcdonaldscanada.csv done


877it [06:41,  2.18it/s]                         

mcdonaldscanada 總共 844 篇圖片已下載


In [4]:
# 抓取貼文內容--------------------
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=334,778")
browser = webdriver.Chrome(options=chrome_options)
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("_blood_mango_")
password_input.send_keys("wangzeng1")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(15)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 


for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    # insHome = pd.read_csv('Home_'+subUrl+'.csv')
    insHome = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    with tqdm(total=insHome.shape[0]) as pbar:
        counter = 0
        for index, row in insHome.iterrows(): #< 922
            if '/p/' in row['url'] and counter > 406 and counter < 923:
                browser.get(url + row['url'])
                time.sleep(10)
                html = browser.page_source
                soup = Soup(html, 'lxml')
                img_elem = soup.select('img.x5yr21d')
                text_elem = soup.select('h1._ap3a')

                insHome.at[index, 'imgSrc'] = img_elem[1]['src']
                
                    if '/reel/' in elem['href'] and imgName.shape[0] > 0:
                        img_elem = elem.select('div div img')
                        imgUrl = img_elem[0]['src']
                        if not os.path.exists(subUrl+'_img'):
                            os.makedirs(subUrl+'_img')
                        with open(subUrl+'_img/'+imgName.values[0]+'.jpg', 'wb') as f:
                            try :
                                f.write(requests.get(imgUrl).content)
                                download_counter = download_counter+1
                            except:
                                print('Failed to download' + imgName+'.jpg')
                        postList.append(elem['href'])
                    pbar.update(1)
            pbar.update(1)
            counter = counter+1
            
    insHome['imgName'] = subUrl+'_'+insHome.index.astype(str)
    insHome.to_csv('Done_'+subUrl+'.csv', index = False)

parsing sonicdrivein.csv
parsing sonicdrivein.csv done


100%|██████████| 2240/2240 [14:36<00:00,  2.56it/s] 


### Data Preprocessing

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

print(device)
torch.cuda.current_device()

2.4.1+cu124
True
cuda


0

In [3]:
torch.cuda.current_device()

0

In [52]:
from transformers import RobertaConfig

# 加载模型配置
config = RobertaConfig.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
vocab_size = config.vocab_size
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 50265


In [48]:
pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emoji", device=0)
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
test_token = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
# 将标题字段转换为tokens
def tokenize_bert(words):
    # 使用BERT Tokenizer对标题进行tokenize
    # tokens = tokenizer(words, truncation=True, padding='max_length', return_tensors='pt')
    tokens = test_token(words, truncation=True, padding='max_length', return_tensors='pt')
    return tokens['input_ids'].tolist()[0]

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [97]:
text = "👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖😜🐱‍🏍🐱‍👤🤳🎂🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha"
vocab_size = 50265  # 词汇表大小
embedding_dim = 768  # 嵌入维度，与你的图像嵌入维度相同
print(tokenizer.encode(text))
text_embedding = nn.Embedding(vocab_size, embedding_dim)
text_embeddings = text_embedding(torch.tensor(tokenizer.encode(text)))  # text_tokens 是文本标记的 tensor
text_embeddings.shape

[0, 31193, 2023, 6569, 10674, 8210, 18636, 9264, 18636, 10172, 31193, 8384, 6569, 10470, 18164, 17, 8384, 38718, 7471, 12605, 6569, 10470, 18164, 17, 8384, 38718, 9264, 12605, 6569, 10470, 18400, 17, 8384, 38718, 7471, 12605, 6569, 10470, 18400, 17, 8384, 38718, 9264, 12605, 39817, 14285, 6569, 10470, 17772, 18636, 23171, 18636, 12736, 6569, 12736, 19002, 18636, 7258, 6569, 10659, 25448, 18636, 48, 6569, 16948, 15389, 17, 8384, 6569, 9357, 8384, 6569, 16948, 15389, 17, 8384, 31193, 10470, 6569, 10470, 15264, 6569, 12736, 9264, 6569, 12736, 23171, 6569, 14285, 9253, 6569, 10659, 13859, 31193, 9357, 6569, 16948, 15389, 17, 8384, 6569, 10659, 2023, 6569, 16948, 15389, 17, 8384, 6569, 16948, 23171, 6569, 16948, 15389, 17, 8384, 31193, 9085, 39817, 10674, 31193, 7471, 18636, 862, 39817, 11423, 18636, 27819, 6569, 10470, 10674, 6569, 10470, 7258, 6569, 12736, 10172, 6569, 4958, 7258, 2489, 2489, 2]


torch.Size([143, 768])

In [49]:
text = "👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖😜🐱‍🏍🐱‍👤🤳🎂🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha"
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))
print("=====================================")
print(test_token.encode(text))
print(test_token.decode(test_token.encode(text)))

[0, 31193, 2023, 6569, 10674, 8210, 18636, 9264, 18636, 10172, 31193, 8384, 6569, 10470, 18164, 17, 8384, 38718, 7471, 12605, 6569, 10470, 18164, 17, 8384, 38718, 9264, 12605, 6569, 10470, 18400, 17, 8384, 38718, 7471, 12605, 6569, 10470, 18400, 17, 8384, 38718, 9264, 12605, 39817, 14285, 6569, 10470, 17772, 18636, 23171, 18636, 12736, 6569, 12736, 19002, 18636, 7258, 6569, 10659, 25448, 18636, 48, 6569, 16948, 15389, 17, 8384, 6569, 9357, 8384, 6569, 16948, 15389, 17, 8384, 31193, 10470, 6569, 10470, 15264, 6569, 12736, 9264, 6569, 12736, 23171, 6569, 14285, 9253, 6569, 10659, 13859, 31193, 9357, 6569, 16948, 15389, 17, 8384, 6569, 10659, 2023, 6569, 16948, 15389, 17, 8384, 6569, 16948, 23171, 6569, 16948, 15389, 17, 8384, 31193, 9085, 39817, 10674, 31193, 7471, 18636, 862, 39817, 11423, 18636, 27819, 6569, 10470, 10674, 6569, 10470, 7258, 6569, 12736, 10172, 6569, 4958, 7258, 2489, 2489, 2]
<s>👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖😜🐱‍🏍🐱‍👤🤳🎂🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha</s>
[101, 100, 5292, 5292

In [50]:
type(test_token.encode(text))

list

In [21]:
print(len(tokenize_bert(text)))
print(len(tokenize_bert("text")))

512
512


In [19]:
print(len(tokenize_bert(text)))
print(len(tokenize_bert("text")))

143
3


In [14]:
from swin_transformer_v2 import SwinTransformerV2
from swin_transformer_v2 import swin_transformer_v2_t
from PIL import Image
from torchvision import transforms
# 載入預訓練的 SwinTransformerV2 模型
SwinTransformerV2 = swin_transformer_v2_t(in_channels=3,
                                          window_size=8,
                                          input_resolution=(256, 256),
                                          sequential_self_attention=False,
                                          use_checkpoint=False)

class SwinFeatures(nn.Module):
    def __init__(self):
        super(SwinFeatures, self).__init__()
        # 取出 ResNet-50 的前面部分，不包含最後的全連接層
        # self.features = nn.Sequential(*list(model.children())[:-1])
        self.features = nn.Sequential(*list(SwinTransformerV2.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x


# 創建一個新的模型實例
feature_extractor = SwinFeatures()
feature_extractor.eval()  # 設置為評估模式

# 定義提取特徵的方法
def extract_features(image_path):
    # 加載並預處理圖像
    image = Image.open(image_path)
    # 確保圖像是RGB格式的
    image = image.convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)  # 添加批次維度

    # 通過新的模型提取特徵
    with torch.no_grad():
        features = feature_extractor(image)

    return features

# 以示例圖像路徑調用提取特徵的函數
image_path = './wendys_img/wendys_0.jpg'
features = extract_features(image_path)

# 檢視提取的特徵的形狀
print("Extracted features shape:", features.shape)

Extracted features shape: torch.Size([1, 96, 64, 64])


In [43]:
insFilter = pd.read_csv('./Original_File/Filter_mcdonalds_switzerland.csv')
#drop if isMeme is -2
print(insFilter.shape)
insFilter = insFilter[insFilter['isMeme'] != -2]
print(insFilter.shape)
insFilter.to_csv('Filter_mcdonalds_switzerland.csv', index = False)

(2087, 3)
(1129, 3)


In [23]:
insFilter['isMeme']==1

0       False
1       False
2       False
3       False
4       False
        ...  
2224    False
2225    False
2226    False
2227    False
2228    False
Name: isMeme, Length: 2229, dtype: bool

In [27]:
subUrlList = ['wendys','mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada','sonicdrivein']

for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    insFilter = pd.read_csv('./Original_File/Filter_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    
    insFilter['isMeme'] = insFilter['isMeme'].apply(lambda x: 1 if x == 1 else(0.5 if x == 0 else 0))
    insFilter.rename(columns = {'imgAlt':'caption', 'imgName':'image_id', 'isMeme':'funny_score'}, inplace = True)
    insFilter.to_csv('Filter_'+subUrl+'.csv', index = False)

parsing wendys.csv
parsing wendys.csv done
parsing mcdonalds.csv
parsing mcdonalds.csv done
parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done
parsing mcdonaldscanada.csv
parsing mcdonaldscanada.csv done
parsing sonicdrivein.csv
parsing sonicdrivein.csv done


insFilter

In [100]:
def text_extraction(data, imgPath):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emoji", device=0)
    tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
    vocab_size = 50265  # 词汇表大小
    embedding_dim = 1  # 嵌入维度，与你的图像嵌入维度相同
    text_embedding = nn.Embedding(vocab_size, embedding_dim).to(device)

    # 将标题字段转换为tokens
    def tokenize_bert(words):
        # 使用BERT Tokenizer对标题进行tokenize
        tokens = tokenizer(words, truncation=True, padding='max_length', max_length=768,return_tensors='pt')
        output = text_embedding(tokens['input_ids'].to(device))
        return output.squeeze(-1).tolist()[0]

    new_data = pd.DataFrame()
    new_data['text'] = data['caption'].apply(tokenize_bert)
    new_data = new_data['text'].apply(pd.Series)
    new_data.columns = ['text_{}'.format(i) for i in range(new_data.shape[1])]
    new_data['image_id'] = data['image_id'].apply(lambda x: imgPath + str(x) + '.jpg')
    new_data['funny_score'] = data['funny_score']

    return new_data

In [94]:
insFilter = pd.read_csv('./Filter_wendys.csv')

In [95]:
imgPath = '../Data/Instagram/' + 'wendys' + '_img/'
text_extraction(insFilter, imgPath)

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,text_0,text_1,text_2,text_3,text_4,text_5,text_6,text_7,text_8,text_9,...,text_760,text_761,text_762,text_763,text_764,text_765,text_766,text_767,image_id,funny_score
0,0.793883,0.069545,-1.311447,-1.218814,-1.598746,0.494994,-0.592103,0.067793,0.270119,0.251997,...,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,../Data/Instagram/wendys_img/wendys_0.jpg,0
1,0.793883,-0.610064,-0.248980,-0.844052,-0.413306,-0.389745,-1.243862,-0.761261,0.127512,-0.170472,...,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,../Data/Instagram/wendys_img/wendys_1.jpg,1000
2,0.793883,-0.912282,-0.259553,0.012632,-0.755190,1.165059,-0.024372,0.966067,-0.154293,-0.111463,...,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,../Data/Instagram/wendys_img/wendys_2.jpg,0
3,0.793883,-0.354310,0.712753,1.325158,-0.261939,1.115581,0.036629,0.036629,0.036629,0.036629,...,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,../Data/Instagram/wendys_img/wendys_3.jpg,1000
4,0.793883,-0.463617,-0.605682,-1.524051,1.115581,0.036629,0.036629,0.036629,0.036629,0.036629,...,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,../Data/Instagram/wendys_img/wendys_4.jpg,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,0.793883,-0.258313,0.463739,0.983964,0.590061,0.259517,-0.353957,0.265535,0.547800,0.261284,...,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,../Data/Instagram/wendys_img/wendys_366.jpg,0
363,0.793883,-1.432584,-0.290138,-0.100823,-0.809414,-0.006921,-0.563101,0.491500,1.115581,0.036629,...,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,../Data/Instagram/wendys_img/wendys_367.jpg,1000
364,0.793883,-1.030454,-0.136545,1.523234,0.012632,0.273535,-0.251326,-0.242746,-1.363341,0.491500,...,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,../Data/Instagram/wendys_img/wendys_368.jpg,0
365,0.793883,-1.355395,-0.520674,-0.549285,0.061573,0.936624,1.034284,0.486650,-0.050262,-1.405267,...,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,0.036629,../Data/Instagram/wendys_img/wendys_369.jpg,0


In [73]:
def tokenize_bert(words):
        # 使用BERT Tokenizer对标题进行tokenize
        tokens = tokenizer(words, truncation=True, padding='max_length', max_length=768,return_tensors='pt')
        print(tokens['input_ids'][0][0])
        output = text_embedding(tokens['input_ids'][0])
        print(output.shape)
        return output.tolist()[0]
tokenize_bert("👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖😜🐱‍🏍🐱‍👤🤳🎂🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha")

tensor(0)
torch.Size([768, 768])


[2.336667060852051,
 -0.8162450790405273,
 -0.9627775549888611,
 1.5990947484970093,
 0.20850633084774017,
 -2.2388088703155518,
 -0.7053582668304443,
 0.5998992919921875,
 0.8678225874900818,
 -0.1497759073972702,
 -1.0288333892822266,
 -0.6947267055511475,
 1.4432806968688965,
 -0.2579844295978546,
 1.1499624252319336,
 -0.9382314085960388,
 -0.9461939930915833,
 -0.47772228717803955,
 -0.6708018779754639,
 -0.9225966930389404,
 -0.6209478974342346,
 0.5084257125854492,
 1.399937629699707,
 1.095009207725525,
 -0.19207607209682465,
 1.3399816751480103,
 -1.673210620880127,
 0.38533923029899597,
 0.8337076902389526,
 -1.4710859060287476,
 0.5904214382171631,
 0.5335190296173096,
 0.3730730414390564,
 -0.5703838467597961,
 0.46065425872802734,
 -1.3911923170089722,
 -1.2434923648834229,
 1.1922404766082764,
 0.8702803254127502,
 -0.9051564931869507,
 -0.9686239957809448,
 -0.9533010721206665,
 1.1604946851730347,
 0.14297685027122498,
 0.8358027338981628,
 -1.8194307088851929,
 0.90062

In [183]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emoji", device=0)
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
vocab_size = 50265  # 词汇表大小
embedding_dim = 768  # 嵌入维度，与你的图像嵌入维度相同
text_embedding = nn.Embedding(vocab_size, embedding_dim).to(device)

words = "👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖😜🐱‍🏍🐱‍👤🤳🎂🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha"
tokens = tokenizer(words, truncation=True , padding='max_length', max_length=100, return_tensors='pt')
# print(tokens['input_ids'][0])
output = text_embedding(tokens['input_ids'].to(device))
print(output.shape)

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


torch.Size([1, 100, 768])


In [189]:
import torch
import torch.nn.functional as F

def find_closest_embeddings(x, embedding_matrix, top_k=1):
    # Normalize both the input tensor x and the embedding matrix
    x = F.normalize(x, dim=2)  # Normalize input tensor along feature dimension
    embedding_matrix = F.normalize(embedding_matrix, dim=1)  # Normalize embedding matrix
    
    # Compute cosine similarity between x and embedding matrix
    similarity = torch.matmul(x, embedding_matrix.T)  # Shape: [10, 50265]
    
    # Find top-k closest embeddings for each tensor in x
    top_k_values, top_k_indices = torch.topk(similarity, top_k, dim=2)
    
    return top_k_indices, top_k_values

In [196]:
temp_11 = torch.rand(2,3,4)
temp_11.long()

tensor([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]])

In [208]:
temp_11 = torch.rand(2,3,4).to(device)
text_embedding = nn.Embedding(50, 4).to(device)
top_k_indices, top_k_values = find_closest_embeddings(temp_11, text_embedding.weight)
print(top_k_indices.squeeze(-1))
# top_k_indices.shape
indices = tokenizer.batch_decode(top_k_indices.squeeze(-1))
print(type(indices))

tensor([[39, 39, 20],
        [16, 20, 20]], device='cuda:0')
<class 'list'>


In [209]:
gogo = tokenizer.batch_encode_plus(indices, truncation=True, padding='max_length', max_length=100, return_tensors='pt')

In [212]:
gogo['input_ids'].shape

torch.Size([2, 100])

In [213]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
vocab_size = 50265  # 詞彙表大小

embedding_dim = 768  # 嵌入维度，與你的圖片嵌入维度相同
text_embedding = nn.Embedding(vocab_size, embedding_dim)

text ="15 fdsfd f fd d  dfsdf"

tokens = tokenizer(text, truncation=True, padding='max_length', max_length=373, return_tensors='pt')
output = text_embedding(tokens['input_ids'])
print(output.shape)

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


torch.Size([1, 373, 768])


In [ ]:
import torch.nn.functional as F
linear = torch.nn.Linear(output.shape[1], 64)
projected_output = linear(output.transpose(1, 2)).transpose(1, 2)
print(projected_output.shape) # torch.Size([1, 32, 768])

In [ ]:
import tqdm

In [215]:
wordList = []
total = 0
dirPath = '..//Oxford_HIC/oxford_hic_data.csv'
data = pd.read_csv(dirPath)

with tqdm(total=data.shape[0]) as pbar:
    for i in range(data.shape[0]):
        for word in str(data['caption'][i]).split():
            if word not in wordList:
                wordList.append(word)
                total += 1
        pbar.update(1)


print(len(wordList))        
print(total)

C:\Users\USER\AppData\Local\Temp\ipykernel_13264\2139088998.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dirPath)
100%|██████████| 3657847/3657847 [8:17:15<00:00, 122.60it/s]   

1226242
1226242


In [217]:
subUrlList = ['wendys','mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada','sonicdrivein']

for subUrl in subUrlList:
    dirPath = './Filter_' + subUrl + '.csv'
    data = pd.read_csv(dirPath)
    
    for i in range(len(data['caption'])):
        for word in str(data['caption'][i]).split():
            if word not in wordList:
                wordList.append(word)
                total += 1
                
print(len(wordList))        
print(total)

1231178
1231178


# Remove Nan from Oxford

In [4]:
import pandas as pd
dirPath = '../Oxford_HIC/Original_File/oxford_hic_data.csv'
data = pd.read_csv(dirPath)
print(data.shape)
data = data.dropna()
print(data.shape)
data = data.reset_index(drop=True)
data.to_csv('../Oxford_HIC/oxford_hic_data.csv', index = False)


C:\Users\USER\AppData\Local\Temp\ipykernel_12588\176095774.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dirPath)


(3657847, 3)
(3653781, 3)


In [6]:
data


,image_id,caption,funny_score
0,bokete_0,My driver's license photo,0.0
1,bokete_1,Refugee relief.,0.0
2,bokete_2,Now! I think I stepped on a cat! What? Really?...,0.0
3,bokete_3,You wouldn't know I was reading a comic book.,0.0
4,bokete_4,"Oh no! I forgot my ・・・・ clothes!""",0.0
...,...,...,...
3653776,british-high-school-honeybee,on time,0
3653777,british-high-school-honeybee,quiz tomorrow bee ready!!!,0
3653778,british-high-school-honeybee,i'll die no matter what my fucking defenses bitch,0
3653779,british-high-school-honeybee,you dont scare me biker dude,0


In [2]:
import pandas as pd
from extractor import addImagePath
import tqdm
import os
dirPath = '../Oxford_HIC/Original_File/oxford_hic_data.csv'
imgPath = '../Oxford_HIC/oxford_img/'
data = pd.read_csv(dirPath)
print(data.shape)
new_data = addImagePath(data, imgPath)
# if image is not exist, remove the row
with tqdm.tqdm(total=data.shape[0]) as pbar:
    for i in range(data.shape[0]):
        if not os.path.exists(data['image_id'][i]):
            data = data.drop(i)
        pbar.update(1)
print(data.shape)
data = data.reset_index(drop=True)
# data.to_csv('../Oxford_HIC/oxford_hic_data.csv', index=False)



C:\Users\USER\AppData\Local\Temp\ipykernel_15004\3786780886.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dirPath)


(3657847, 3)


 89%|████████▊ | 3243763/3657847 [01:48<00:13, 30014.97it/s]


KeyboardInterrupt: 

In [2]:
data.to_csv('../Oxford_HIC/Filter_oxford_hic_data.csv', index=False)

In [4]:
print(3653781 - 3400257)

253524


In [41]:
import pandas as pd
dirPath = '../Oxford_HIC/Filtered_oxford_hic_data.csv'
imgPath = '../Oxford_HIC/oxford_img/'

data = pd.read_csv(dirPath)
# data
# data['image_id'] = data['image_id'].apply(lambda x: x.split('/')[-1].split('.')[0])
data["funny_score"] = data["funny_score"].apply(lambda x: int(float(x.replace(',', ''))) if type(x) == str else int(float(x)))
data["funny_score"] = data["funny_score"]/data["funny_score"].max()

C:\Users\USER\AppData\Local\Temp\ipykernel_15004\3302334520.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dirPath)


In [44]:
data.to_csv('../Oxford_HIC/Filtered_oxford_hic_data.csv', index=False)

In [4]:
import pandas as pd
dirPath = '../Oxford_HIC/Filtered_oxford_hic_data.csv'
data = pd.read_csv(dirPath)
# remove caption is imgflip_118
print(data.shape)
data = data[data['image_id'] != 'imgflip_118']
print(data.shape)

(3400257, 3)
(3398346, 3)


In [5]:
data.to_csv('../Oxford_HIC/Filtered_oxford_hic_data.csv', index=False)

In [6]:
import pandas as pd
dirPath = '../Oxford_HIC/Filtered_oxford_hic_data.csv'
data = pd.read_csv(dirPath)
# remove caption is imgflip_118
print(data.shape)

(3398346, 3)
